In [2]:
from pypdf import PdfReader
from pypdf.errors import PdfReadError
from transformers import pipeline
import pandas as pd
import torch
import sentencepiece
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk


In [4]:
#nltk.download('punkt') # only uncomment if not already downloaded
#nltk.download('stopwords')

In [9]:
ArticleToSummarize = "pdf_database\pdf3.pdf"

def pdfTE(pdfFile, version=1, start=0, end=0): # Function for text extraction from PDF. # Version states which version to use, with arguments(start, end) being the starting page and ending page.
    if version==1:
        with open(pdfFile, "rb") as file: # Read in binary to handle breakline statements better (\n)
            pdfReader = PdfReader(file)
            for page in pdfReader.pages:
                yield page.extract_text() # Use of generator as keeping the whole article in the memory results in memory error.
    else:
        with open(pdfFile, "rb") as file: # Read in binary to handle breakline statements better (\n)
            pdfReader = PdfReader(file)
            if end==0:
                end=len(pdfReader.pages)-1
            for num in range(start,end): # Iterator over page (num) 
                yield pdfReader.pages[num].extract_text() # Text extraction of page (num)
            
def articleC(pdfFile, version=1,start=0,end=0): # Function to return the article in one string # Version and start, end arguments necessary for pdfTE integration.
    if version==1:
        textCombiner = pdfTE(pdfFile)
    else:
        textCombiner = pdfTE(pdfFile,2,start,end) # Choosing for version 2 if sumArticle2 failed on version 1
    textCombined = ""
    for text in textCombiner: # Loop over generator object to sum text of pages into one string
        textCombined += text
    if textCombined == "":
        raise Exception("Oops! The task failed as the PDF file is empty.") # Exception that handles empty files
    elif len(textCombined) < 10:
        raise Exception("Oops! The task failed as the PDF file has too little characters.") # Exception that handles files with too little characters for a summary
    else:
        return textCombined

def sumArticle2(pdfFile): # Function to summarize article as a whole
    if pdfFile.endswith(".pdf"): # First check if file extension is ".pdf" format.
        try:
            summarizer = pipeline("summarization", model="pszemraj/led-large-book-summary") # Model used from the huggingface hub (https://huggingface.co/pszemraj/led-large-book-summary)
            articleCombined = articleC(pdfFile) # Iterate over text combiner function
            summarizedPage = summarizer(articleCombined,max_length=1000, min_length=200, do_sample=True) # Max/min length for length of summarization in characters.
            return summarizedPage[0]["summary_text"] # Return summary as string
        except PdfReadError as PRE: # Second check if file extension is ".pdf" format when first check fails.
            return f"Oops! a PDF Read Error {PRE} happened. Please retry the task once the issue has been resolved."
        except OSError as OS: # Exception handling of OS errors, as PdfReadError doesn't catch all file extension errors.
            return f"Oops! an OS Error {OS} happened. Please retry the task once the issue has been resolved."
        except (RuntimeError, IndexError) as RIE: # Exception check if pdf fails to be summarized due to it having too many tokens (> 16384)
            print(f"Oops! {RIE}, The file was too big to configure.")
            try:
                amountPages = int(input("Per how many pages would you like a summary? ")) # Asking user input to make the summary/ies on.
                print("Warning, this summary may take a while to be produced.") # Give feedback to the user that because of the large pdf, the models computation will be larger
                if amountPages == 1: # If the user wants to get a summary per page, the calculation of the articleC function needs to be changed.
                    end = amountPages
                else:
                    end = amountPages-1 # If the user wants to get a summary per an amount of pages, the calculation needs to be the standard one.
                start = 0
                summary = "" # Empty string to merge all the separate summaries in
                while(end<len(PdfReader(ArticleToSummarize).pages)): # Loop till all the pages have been summarized (#* May need to implement check that last iteration is at the last page in future)
                    articleCombined = articleC(pdfFile,2,start,end)
                    if amountPages == 1:
                        summarizedPage = summarizer(articleCombined,max_length=200, min_length=50, do_sample=False) # Summarizer for per page, different max/min length as per page has less characters in one iteration than per amount of pages, gives warning if min/max is too little.
                    else:
                        summarizedPage = summarizer(articleCombined,max_length=1000, min_length=200, do_sample=False) # Max/min length for length of summarization  per amount of pages in characters.
                    summary += f"Summary of pages {start+1}-{end+1}\n" + summarizedPage[0]["summary_text"] + '\n'   #* May need to implement different message in case of summarization per page.            
                    start += amountPages # Calculations to determine next start/end point of iteration.
                    end += amountPages
                return summary
            except ValueError as VE: # Exception that handles user input.
                return f"Oops! {VE}, You didn't give an integer."
            except IndexError as IE: # Exception that handles user input.
                return f"Oops! {IE}, you gave an integer larger than the actual amount of pages in the paper."
            except Exception as E: # Exception that handles all different errors and asks user to feedback the error to the devs to enhance the model.
                return f"Oops! An unexpected error occured, {E}. Please report the error to the team."
    else:
        raise Exception("The file format is not a valid pdf.")

def pdfKE(pdfFile, language='english'): # Function to extract keywords from the PDF, language input needed as language needs to be part of stopwords folder. Standard keyword is english
    if pdfFile.endswith(".pdf"): # First check if file extension is ".pdf" format.
        try:
            articleCombined = articleC(pdfFile).lower() # Iterate over generator
            tokens = word_tokenize(articleCombined) # Tokenize all words in the article
            punctuations = ["(",")",";",":","[","]",",","!","=","==","<",">","@","#","$","%","^","&","*",".","//","{","}","...","``","+","\'\'","-","~","\"","’",]
            stopWords = stopwords.words(f'{language}')
            keywords = [word for word in tokens if word not in stopWords and word not in punctuations] # Filter the words so that mostlikely keywords will be extracted
            keywordExtracted = pd.Series(keywords).value_counts().index[:5] # Keywords formatting as a list
            keywordString = " ".join(keywordExtracted) # Keywords formatting as a string
            keywordDict = {i+1:keywordExtracted[i] for i in range(5)}
            return keywordDict
        except PdfReadError as PRE: # Second check if file extension is ".pdf" format when first check fails.
            return f"Oops! a PDF Read Error {PRE} happened. Please retry the task once the issue has been resolved."
        except OSError as OS: # Exception handling of OS errors, as PdfReadError doesn't catch all file extension errors.
            return f"Oops! an OS Error {OS} happened. Please retry the task once the issue has been resolved."
    else:
        return f"Oops! The task failed. Please retry the task once the issue has been resolved." # Message if program fails to execute

32


In [10]:
# Summarize text
text = sumArticle2(ArticleToSummarize)
print(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (28510 > 16384). Running this sequence through the model will result in indexing errors


Oops! index out of range in self, The file was too big to configure.


Your max_length is set to 1000, but your input_length is only 361. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=180)


Done 1


Your max_length is set to 1000, but your input_length is only 846. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=423)
Your max_length is set to 1000, but your input_length is only 702. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=351)


KeyboardInterrupt: 

In [21]:
# Extract keywords and output them in dictionary format
keywords = pdfKE(ArticleToSummarize)
print(keywords)

{1: 'stanley', 2: 'vehicle', 3: 'figure', 4: '/h20850', 5: 'online'}


In [13]:
# Summarization translation
translation = pipeline("translation", model="Helsinki-NLP/opus-mt-en-nl") # Model used for the translation, imported from Huggingface (https://huggingface.co/Helsinki-NLP/opus-mt-en-nl)
translatedText = translation(text)[0]['translation_text']
print(translatedText)

In dit document tonen Pillai en Nair aan dat een nieuw algoritmisch model voor het voorspellen van de softwarekosten en andere beheerparameters verbeterd kan worden ten opzichte van het Putnam SLIM-model. Ze tonen ook aan dat het nieuwe model betrouwbaarder is dan het vorige. De auteurs bespreken kort de verschillende modellen die gebruikt worden om de "softwareproductiviteit" te voorspellen en hoe ze van elkaar verschillen. Ze bespreken de verschillende manieren waarop verschillende modellen kunnen worden vergeleken en concluderen dat het putnammodel het meest effectief is voor het voorspellen van de mankrachtvereisten tijdens de vroege stadia van een project en voor het voorspellen van toekomstige prestaties. Daarnaast bespreken ze verschillende soorten voorspellingsinstrumenten die kunnen worden gebruikt om de snelheid en nauwkeurigheid van een bepaald model te schatten. Een voorbeeld van een van deze instrumenten is het "routinely run-down gewogen gemiddelde," dat de snelheid meet 

In [9]:
# Cell only works in environment that works on pip. Evaluate isn't a library that has a conda recipe.

import evaluate

metric = evaluate.load("sacrebleu") # Model generated translation evaluated with DeepL generated translation

evaluationText = "In dit artikel worden verschillende soorten modellen besproken die worden gebruikt om de ontwikkelingstijd en -kosten van software te voorspellen. Het meest gebruikte model is het Putnam SLIM model, dat gebaseerd is op een hellings-leercurve. De auteurs zijn echter van mening dat dit model niet betrouwbaar is bij het voorspellen van zeer vroege stadia van een project. Ze gebruiken een andere benadering, het 'Putnam-model', om toekomstige prestaties te voorspellen. In dit artikel stellen ze een nieuw type model voor dat toekomstige prestaties veel nauwkeuriger voorspelt dan het vorige Putnam-model. Dit nieuwe model kan op verschillende manieren worden geïmplementeerd, waaronder: het correleren van statistieken van real-time experimenten met machine learning, het gebruik van statistische methoden zoals rinterpreter en correlatieanalyse, modellering door directe observatie en het gebruik van geavanceerde statistische technieken zoals gewogen gemiddelden. De auteurs bespreken ook verschillende benaderingen voor het voorspellen van toekomstige prestaties van een bepaald soort product of functie. Als een bedrijf bijvoorbeeld een nieuw stuk machine zou introduceren, zou het in staat moeten zijn om het snel en op een hoog niveau te produceren over een lange periode."

translatedText = "Het meest gebruikte model is het Putnam SLIM model, dat gebaseerd is op een hellings-learning curve. Echter, de auteurs geloven dat het niet betrouwbaar is in het voorspellen van zeer vroege stadia van een project. Ze gebruiken een andere aanpak genaamd het 'Putnam model' om toekomstige prestaties te voorspellen. In dit document suggereren ze een nieuw type model dat toekomstige prestaties veel nauwkeuriger voorspelt dan het vorige Putnam model. Dit nieuwe model kan worden toegepast op verschillende manieren, waaronder: het correleren van statistieken van real-time experimenten met machine learning, gebruik makend van statistische methoden zoals rinterpreter en correlatieanalyse, modelleren door middel van directe observatie, en gebruik makend van geavanceerde statistische technieken zoals gewogen gemiddelde. De auteurs bespreken ook verschillende benaderingen voor het voorspellen van toekomstige prestaties van een bepaald soort product of functie. Bijvoorbeeld, als een bedrijf een nieuw stuk machines zou moeten kunnen introduceren, zou het snel en op een hoog niveau van tijd moeten produceren."

predictions = [
    translatedText
]
references = [
    [
        evaluationText
    ]
]
metric.compute(predictions=predictions, references=references)

### ==============================
#{'score': 53.86425158362457,
#'counts': [149, 112, 95, 80],
#'totals': [173, 172, 171, 170],
#'precisions': [86.1271676300578,
# 65.11627906976744,
# 55.55555555555556,
# 47.05882352941177],
#'bp': 0.8704644809074915,
#'sys_len': 173,
#'ref_len': 197}
### ===============================

ImportError: DLL load failed while importing lib: Kan opgegeven procedure niet vinden.